In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
from pathlib import Path
import colour
import matplotlib.pyplot as plt
import numpy as np
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from dhdrnet import image_loader as il

# Try and load and fuse some of the basic HDR Images

In [ ]:
DATA_DIR = Path("../../data/")

num_imgs = 5
loaded_images, image_paths = il.read_images(DATA_DIR / "dngs", limit=num_imgs)

In [ ]:
for p, li in loaded_images.items():
    fig, ax = colour.plotting.plot_image(li.read_data())
    ax.set_title(p)
    plt.show()

## Adjust the exposure
Let's try and adjust the exposures. 
Picking 5 evenly spaced values over [-4,4]

In [ ]:
multi_exposure = il.gen_multi_exposure(loaded_images)
for name, imgs in multi_exposure.items():
    for (md, img) in imgs:
#         fig, ax = colour.plotting.plot_image(img)
        plt.imshow(img)
        ax.set_title(name)
#         plt.show()


In [ ]:
multi_exposure = il.gen_multi_exposure(loaded_images)
colour.write_image(list(multi_exposure.values())[0][1],"./out.png",bit_depth="float32")
colour.write_image(list(multi_exposure.values())[0][1],"./out.bmp",bit_depth="float32")

In [ ]:
# What about writing the images out to disk now?
il.write_multi_exposure(image_paths)

In [ ]:
mexp_path_dict = il.group_ldr_paths(DATA_DIR / "processed")
merged = il.merge_all_test(mexp_path_dict)

In [ ]:
import rawpy
cols = 3
fig, axs = plt.subplots(ncols=cols + 1,nrows=len(merged), squeeze=False, figsize=tuple(5*i for i in (10, 40)))
plt.tight_layout()
print(axs.shape)
for row, (deb,rob,mert) in enumerate(merged):
    name = deb.name.split(".")[0]
    axs[row,0].set_xlabel("debevec " + name)
    axs[row,0].imshow(plt.imread(str(deb)))
    
    axs[row,1].set_xlabel("robertson " + name)
    axs[row,1].imshow(plt.imread(str(rob)))
    
    axs[row,2].set_xlabel("mertens " + name)
    axs[row,2].imshow(plt.imread(str(mert)))

    raw_path = str(DATA_DIR/"dngs"/f"{name}.dng")
    dng = rawpy.imread(raw_path).postprocess()
    axs[row,3].set_xlabel("dng " + name)
    axs[row,3].imshow(dng)
    
    fig.show()